In [2]:
import sys

# SECTION 1: Environment Setup for MongoDB
# Install required packages
!{sys.executable} -m pip install pymongo

# Import libraries
import pymongo

# Verify installation
print("\n--- Verification ---")
print(f"PyMongo version: {pymongo.version}")
print("--- MongoDB Setup Complete ---")


[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



--- Verification ---
PyMongo version: 4.15.5
--- MongoDB Setup Complete ---


In [ ]:
# SECTION 2: MongoDB Atlas Connection

# Define the connection string variable
# IMPORTANT: Replace 'YOUR_MONGODB_CONNECTION_STRING' with your actual MongoDB Atlas connection string.
# Example: 'mongodb+srv://<username>:<password>@<cluster-name>.mongodb.net/?retryWrites=true&w=majority'
MONGODB_URI = "YOUR_MONGODB_CONNECTION_STRING"

# Connect using pymongo
client = None
try:
    client = pymongo.MongoClient(MONGODB_URI)
    print("Successfully connected to MongoDB Atlas.")

    # Test connection with server info or ping
    print("\n--- Connection Test ---")
    print("Server info:")
    print(client.server_info())
    print("--- Connection Test Complete ---")

except pymongo.errors.ConfigurationError as e:
    print(f"Configuration Error: {e}")
    print("Please ensure your MONGODB_URI is correctly formatted and includes your credentials.")
except pymongo.errors.ConnectionFailure as e:
    print(f"Connection Failure: {e}")
    print("Could not connect to MongoDB Atlas. Please check your network connection and MONGODB_URI.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # It's good practice to close the connection when done, though for simple scripts, it might not be strictly necessary.
    # client.close() # Uncomment if you want to explicitly close the connection after this cell runs.
    pass

Successfully connected to MongoDB Atlas.

--- Connection Test ---
Server info:
{'version': '8.0.18', 'gitVersion': '5d1557a9ac6f1cd358cef85fd11c0fabfcfc4e2e', 'modules': ['enterprise'], 'allocator': 'tcmalloc-google', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [8, 0, 18, 0], 'bits': 64, 'debug': False, 'maxBsonObjectSize': 16777216, 'storageEngines': ['devnull', 'inMemory', 'queryable_wt', 'wiredTiger'], 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1769673649, 2), 'signature': {'hash': b'A\xd7\\\x8f\xb6\x12*Q\x9cq\xc1\xfdf\xae\x90\xc4\xa0\x06\x8f\xe4', 'keyId': 7553323988634566657}}, 'operationTime': Timestamp(1769673649, 2)}
--- Connection Test Complete ---


In [ ]:
# SECTION 3: Database & Collection Creation

# Define database and collection names
DATABASE_NAME = "mydatabase"
COLLECTION_NAME = "mycollection"

# Ensure client is connected
if client is not None:
    try:
        # Create a database (implicitly created when you access it and insert data)
        db = client[DATABASE_NAME]
        print(f"\nAccessing database: '{DATABASE_NAME}'")

        # Create a collection (implicitly created when you access it and insert data)
        collection = db[COLLECTION_NAME]
        print(f"Accessing collection: '{COLLECTION_NAME}' in database '{DATABASE_NAME}'")

        # Insert a dummy document to ensure database and collection are created
        # MongoDB creates the database and collection only when data is first inserted.
        dummy_document = {"name": "test_item", "value": 1}
        collection.insert_one(dummy_document)
        print("Inserted a dummy document to ensure database and collection creation.")
        collection.delete_one(dummy_document) # Remove dummy document
        print("Removed dummy document.")

        # List all database names
        print("\n--- Listing Databases ---")
        print("Databases:")
        for db_name in client.list_database_names():
            print(f"- {db_name}")

        # List all collections in the created database
        print(f"\n--- Listing Collections in '{DATABASE_NAME}' ---")
        print(f"Collections in '{DATABASE_NAME}':")
        for coll_name in db.list_collection_names():
            print(f"- {coll_name}")

    except Exception as e:
        print(f"An error occurred during database/collection creation or listing: {e}")
else:
    print("MongoDB client is not connected. Please ensure SECTION 2 ran successfully.")


Accessing database: 'mydatabase'
Accessing collection: 'mycollection' in database 'mydatabase'
An error occurred during database/collection creation or listing: your_mongodb_connection_string:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 697b1130267087ab646a5a0c, topology_type: Unknown, servers: [<ServerDescription ('your_mongodb_connection_string', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('your_mongodb_connection_string:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


In [ ]:
# SECTION 4: Insert Operations

if client is not None and 'db' in locals() and 'collection' in locals():
    try:
        print("\n--- Insert Operations ---")

        # 4.1 Insert one document
        print("\nInserting one document...")
        document_one = {"name": "Alice", "age": 30, "city": "New York"}
        insert_one_result = collection.insert_one(document_one)
        print(f"Inserted one document with _id: {insert_one_result.inserted_id}")

        # 4.2 Insert multiple documents
        print("\nInserting multiple documents...")
        documents_many = [
            {"name": "Bob", "age": 24, "city": "Los Angeles"},
            {"name": "Charlie", "age": 35, "city": "Chicago"},
            {"name": "Diana", "age": 29, "city": "Houston"}
        ]
        insert_many_result = collection.insert_many(documents_many)
        print(f"Inserted {len(insert_many_result.inserted_ids)} documents.")

        # 4.3 Print inserted IDs
        print("\nInserted IDs:")
        print(f"Single document ID: {insert_one_result.inserted_id}")
        print(f"Multiple documents IDs: {insert_many_result.inserted_ids}")

        # 4.4 Fetch and display all inserted data
        print("\nFetching and displaying all data in the collection:")
        for doc in collection.find():
            print(doc)

    except Exception as e:
        print(f"An error occurred during insert operations: {e}")
else:
    print("MongoDB client, database, or collection not properly initialized. Please ensure previous sections ran successfully.")

In [ ]:
# SECTION 5: Read (Query) Operations

if client is not None and 'db' in locals() and 'collection' in locals():
    try:
        print("\n--- Read (Query) Operations ---")

        # 5.1 Find all documents
        print("\n5.1 Finding all documents:")
        for doc in collection.find():
            print(doc)

        # 5.2 Find documents with a filter
        print("\n5.2 Finding documents with a filter (age > 30):")
        for doc in collection.find({"age": {"$gt": 30}}):
            print(doc)

        # 5.3 Find one document
        print("\n5.3 Finding one document (name = 'Alice'):")
        one_doc = collection.find_one({"name": "Alice"})
        print(one_doc)

        # 5.4 Use projection (select fields)
        print("\n5.4 Finding documents with projection (name and city only):")
        for doc in collection.find({}, {"name": 1, "city": 1, "_id": 0}): # _id is included by default, so explicitly exclude it
            print(doc)

        # 5.5 Use comparison operators ($gt, $lt, $in)
        print("\n5.5 Finding documents with comparison operators (age between 25 and 30, cities in ['New York', 'Chicago']):")
        query_complex = {
            "age": {"$gte": 25, "$lte": 30},
            "city": {"$in": ["New York", "Chicago"]}
        }
        for doc in collection.find(query_complex):
            print(doc)

    except Exception as e:
        print(f"An error occurred during read operations: {e}")
else:
    print("MongoDB client, database, or collection not properly initialized. Please ensure previous sections ran successfully.")

In [ ]:
# SECTION 6: Update Operations

if client is not None and 'db' in locals() and 'collection' in locals():
    try:
        print("\n--- Update Operations ---")

        # 6.1 Update one document
        print("\n6.1 Updating one document (Alice's city):")
        # Before update
        print("Before update:")
        print(collection.find_one({"name": "Alice"}))

        collection.update_one(
            {"name": "Alice"},
            {"$set": {"city": "San Francisco"}}
        )
        print("After update:")
        print(collection.find_one({"name": "Alice"}))

        # 6.2 Update multiple documents
        print("\n6.2 Updating multiple documents (add status 'active' to age > 25):")
        # Before update
        print("Before update (age > 25):")
        for doc in collection.find({"age": {"$gt": 25}}):
            print(doc)

        collection.update_many(
            {"age": {"$gt": 25}},
            {"$set": {"status": "active"}}
        )
        print("After update (age > 25):")
        for doc in collection.find({"age": {"$gt": 25}}):
            print(doc)

        # 6.3 Use $inc operator
        print("\n6.3 Using $inc operator (increment Bob's age by 1):")
        # Before update
        print("Before $inc:")
        print(collection.find_one({"name": "Bob"}))

        collection.update_one(
            {"name": "Bob"},
            {"$inc": {"age": 1}}
        )
        print("After $inc:")
        print(collection.find_one({"name": "Bob"}))

    except Exception as e:
        print(f"An error occurred during update operations: {e}")
else:
    print("MongoDB client, database, or collection not properly initialized. Please ensure previous sections ran successfully.")

In [ ]:
# SECTION 7: Delete Operations

if client is not None and 'db' in locals() and 'collection' in locals():
    try:
        print("\n--- Delete Operations ---")

        # 7.1 Delete one document
        print("\n7.1 Deleting one document (Diana):")
        # First, ensure Diana is in the collection (from previous insert operation)
        print(f"Documents before deleting Diana: {collection.count_documents({})}")
        print(f"Diana's document before deletion: {collection.find_one({"name": "Diana"})}")

        delete_one_result = collection.delete_one({"name": "Diana"})
        print(f"Deleted {delete_one_result.deleted_count} document.")

        print(f"Documents after deleting Diana: {collection.count_documents({})}")
        print(f"Diana's document after deletion: {collection.find_one({"name": "Diana"})}")

        # 7.2 Delete multiple documents
        print("\n7.2 Deleting multiple documents (age < 30):")
        # Insert some new temporary documents for multi-delete to demonstrate effectively
        temp_docs = [
            {"name": "Frank", "age": 22, "city": "Dallas"},
            {"name": "Grace", "age": 28, "city": "Seattle"},
            {"name": "Heidi", "age": 32, "city": "Miami"} # This one should not be deleted by the filter
        ]
        collection.insert_many(temp_docs)
        print("Inserted temporary documents for multi-delete demo.")

        print(f"Documents before deleting age < 30: {collection.count_documents({})}")
        print("Documents with age < 30 before deletion:")
        for doc in collection.find({"age": {"$lt": 30}}):
            print(doc)

        delete_many_result = collection.delete_many({"age": {"$lt": 30}})
        print(f"Deleted {delete_many_result.deleted_count} documents.")

        print(f"Documents after deleting age < 30: {collection.count_documents({})}")
        print("Remaining documents with age < 30 after deletion:")
        for doc in collection.find({"age": {"$lt": 30}}):
            print(doc)

        # Show all remaining documents
        print("\nAll remaining documents in the collection:")
        for doc in collection.find():
            print(doc)

    except Exception as e:
        print(f"An error occurred during delete operations: {e}")
else:
    print("MongoDB client, database, or collection not properly initialized. Please ensure previous sections ran successfully.")

In [ ]:
# SECTION 8: Advanced Practical Queries

if client is not None and 'db' in locals() and 'collection' in locals():
    try:
        print("\n--- Advanced Practical Queries ---")

        # Ensure we have some data for advanced queries, re-insert if necessary
        # (The delete operations in Section 7 might have removed some previously inserted data)
        initial_docs = [
            {"name": "Alice", "age": 30, "city": "San Francisco", "status": "active"},
            {"name": "Bob", "age": 25, "city": "Los Angeles", "status": "active"},
            {"name": "Charlie", "age": 35, "city": "Chicago", "status": "active"},
            {"name": "David", "age": 28, "city": "New York"},
            {"name": "Eve", "age": 40, "city": "London"}
        ]
        # Clear existing and insert fresh data to have a consistent state for this section
        collection.delete_many({})
        collection.insert_many(initial_docs)
        print("Re-inserted initial documents for advanced queries.")

        # 8.1 Sort documents
        print("\n8.1 Documents sorted by age (ascending):")
        for doc in collection.find().sort("age", pymongo.ASCENDING):
            print(doc)

        print("\n8.1 Documents sorted by age (descending) and then name (ascending):")
        for doc in collection.find().sort([("age", pymongo.DESCENDING), ("name", pymongo.ASCENDING)]):
            print(doc)

        # 8.2 Limit results
        print("\n8.2 Documents limited to 2 results (after sorting by age ascending):")
        for doc in collection.find().sort("age", pymongo.ASCENDING).limit(2):
            print(doc)

        # 8.3 Count documents
        print("\n8.3 Counting documents:")
        total_documents = collection.count_documents({})
        print(f"Total documents in collection: {total_documents}")

        documents_in_new_york = collection.count_documents({"city": "New York"})
        print(f"Documents where city is 'New York': {documents_in_new_york}")

        # 8.4 Use indexes
        print("\n8.4 Creating and using indexes:")
        print("Creating index on 'name' field...")
        collection.create_index("name") # Can be a string for single field ascending index
        print("Index created. Listing current indexes:")
        for index in collection.list_indexes():
            print(index)

        # Query that will use the index
        print("\nQuerying using the 'name' index (name = 'Alice'):")
        for doc in collection.find({"name": "Alice"}):
            print(doc)

        # 8.5 Explain index usage briefly via code
        print("\n8.5 Explaining index usage:")
        explain_result = collection.find({"name": "Alice"}).explain()
        print("Query Explanation (briefly):")
        # The exact structure of explain output can vary, focus on key indicators
        if 'queryPlanner' in explain_result and 'winningPlan' in explain_result['queryPlanner']:
            winning_plan = explain_result['queryPlanner']['winningPlan']
            print(f"Winning Plan stage: {winning_plan['stage']}")
            if 'inputStage' in winning_plan and 'indexName' in winning_plan['inputStage']:
                print(f"Index used: {winning_plan['inputStage']['indexName']}")
            elif winning_plan['stage'] == 'IXSCAN':
                 print("Index scan (IXSCAN) performed.")
            else:
                print("No specific index mentioned in winning plan inputStage, but queryPlanner was present.")
        else:
            print("Could not retrieve detailed winning plan information from explain output.")

    except Exception as e:
        print(f"An error occurred during advanced query operations: {e}")
else:
    print("MongoDB client, database, or collection not properly initialized. Please ensure previous sections ran successfully.")

In [ ]:
# SECTION 9: Realistic Mini Practice

import datetime

if client is not None:
    try:
        print("\n--- SECTION 9: Realistic Mini Practice ---")

        # Define a new collection for users
        USERS_COLLECTION_NAME = "users"
        users_collection = client[DATABASE_NAME][USERS_COLLECTION_NAME]
        print(f"\nUsing collection: '{USERS_COLLECTION_NAME}'")

        # Clear the collection to ensure a clean start for the practice
        users_collection.delete_many({})
        print(f"Cleared existing documents from '{USERS_COLLECTION_NAME}'.")

        # 9.1 Insert operations for 'users' collection
        print("\n9.1 Inserting sample users...")
        user_data = [
            {"name": "John Doe", "email": "john.doe@example.com", "age": 30, "role": "user", "created_at": datetime.datetime.now()},
            {"name": "Jane Smith", "email": "jane.smith@example.com", "age": 24, "role": "admin", "created_at": datetime.datetime.now()},
            {"name": "Peter Jones", "email": "peter.j@example.com", "age": 45, "role": "user", "created_at": datetime.datetime.now()},
            {"name": "Alice Brown", "email": "alice.b@example.com", "age": 29, "role": "editor", "created_at": datetime.datetime.now()},
            {"name": "Bob White", "email": "bob.w@example.com", "age": 50, "role": "user", "status": "inactive", "created_at": datetime.datetime.now()}
        ]
        insert_results = users_collection.insert_many(user_data)
        print(f"Inserted {len(insert_results.inserted_ids)} users.")

        print("Current users in the collection:")
        for user in users_collection.find():
            print(user)

        # 9.2 Find users above a certain age (e.g., above 30)
        print("\n9.2 Finding users above age 30:")
        for user in users_collection.find({"age": {"$gt": 30}}):
            print(user)

        # 9.3 Update a user's role (e.g., change John Doe's role to 'manager')
        print("\n9.3 Updating John Doe's role to 'manager':")
        # Before update
        print("John Doe before update:")
        print(users_collection.find_one({"name": "John Doe"}))

        users_collection.update_one(
            {"name": "John Doe"},
            {"$set": {"role": "manager"}}
        )
        # After update
        print("John Doe after update:")
        print(users_collection.find_one({"name": "John Doe"}))

        # 9.4 Delete inactive users (e.g., users with status 'inactive')
        print("\n9.4 Deleting inactive users:")
        print(f"Users before deletion: {users_collection.count_documents({})}")
        print("Inactive users before deletion:")
        for user in users_collection.find({"status": "inactive"}):
            print(user)

        delete_result = users_collection.delete_many({"status": "inactive"})
        print(f"Deleted {delete_result.deleted_count} inactive users.")

        print(f"Users after deletion: {users_collection.count_documents({})}")
        print("Remaining users:")
        for user in users_collection.find():
            print(user)

    except Exception as e:
        print(f"An error occurred during realistic mini practice: {e}")
else:
    print("MongoDB client is not connected. Please ensure SECTION 2 ran successfully.")